In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

# Generate dummy data
x_train = np.random.random((100, 100, 100, 3))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
x_test = np.random.random((20, 100, 100, 3))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=32)

Epoch 1/10


In [5]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
#model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(112, 112, 3)))
model.add(Conv2D(32, (5, 5), activation='linear', input_shape=(112, 112, 3)))
model.add(LeakyReLU(alpha=0.01))

model.add(Conv2D(32, (3, 3), activation='linear'))
model.add(LeakyReLU(alpha=0.01))

model.add(MaxPooling2D(pool_size=3))
#model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), activation='linear'))
model.add(LeakyReLU(alpha=0.01))
model.add(Conv2D(64, (3, 3), activation='linear'))
model.add(LeakyReLU(alpha=0.01))
model.add(Conv2D(64, (3, 3), activation='linear'))
model.add(LeakyReLU(alpha=0.01))

model.add(MaxPooling2D(pool_size=3))
#model.add(Dropout(0.25))

model.add(Conv2D(128, (5, 5), activation='linear'))
model.add(LeakyReLU(alpha=0.01))

model.add(Conv2D(128, (3, 3), activation='linear'))
model.add(LeakyReLU(alpha=0.01))

model.add(Conv2D(128, (3, 3), activation='linear'))
model.add(LeakyReLU(alpha=0.01))

model.add(AveragePooling2D(pool_size=3, strides=(3, 3),padding='same'))

#model.add(RMSPooling2D(strides=(3, 3)))
# This should RMSPooling2D
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024, activation='linear'))
model.add(LeakyReLU(alpha=0.01))

# FeaturePoolLayer1
model.add(Dropout(0.5))
model.add(Dense(1024, activation='linear'))
model.add(LeakyReLU(alpha=0.01))

# FeaturePoolLayer2
#chnaged from 10 to 5 because an error 
model.add(Dense(5, activation='softmax'))
# This is indeed relu in the original lasagne implementation, but softmax could be tried as well

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#model.summary()



In [6]:
#Preparing Data Augmentation Configuration 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0,
        zoom_range=[1 / 1.15, 1.15],
        horizontal_flip=True,
        rotation_range=10,
        validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

#i = 0
train_generator = train_datagen.flow_from_directory(
    directory=r"./data/train_data/",
    target_size=(112, 112),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset="training")
#    save_to_dir="D:/Separating Images into subfolders/train augmented"):
#   i += 1
#   if i > 20: # save 20 images
#        break  # otherwise the generator would loop indefinitely

#print(train_generator.class_indices)

valid_generator = train_datagen.flow_from_directory(
    directory=r"./data/train_data/",
    target_size=(112, 112),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset="validation")

test_generator = test_datagen.flow_from_directory(
    directory=r"./data/test_data/",
    target_size=(112, 112),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)



# Training the model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
 
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

# Evaluating the model 
score = model.evaluate_generator(generator=valid_generator)
#print("Lables", model.metrics_names)

#predicting output
#reseting the test generator before calling predict_generator, in order not to get weird output
test_generator.reset()
 
pred=model.predict_generator(test_generator,verbose=1) 
print(pred)
predicted_class_indices=np.argmax(pred, axis=-1) #multiple categories
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

#Saving Results to csv file
filenames=test_generator.filenames 
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

OSError: [Errno 2] No such file or directory: './data/train_data/'